# MovieDB Database Exploration

⚠️ This notebook was generated during a live demo. It is not a complete analysis and may contain errors.

This notebook explores the PostgreSQL moviedb database with comprehensive analysis of:
- Database schema and relationships
- Movies, users, and ratings data
- Data quality assessment
- Business insights and patterns
- Advanced analytics and visualizations


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2
from sqlalchemy import create_engine, text
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline


In [ ]:
# Database connection parameters
# Try different connection options based on environment
import os

# Option 1: Docker compose setup (external)
DB_CONFIGS = [
    {
        'name': 'Docker Compose (localhost:5433)',
        'host': 'localhost',
        'port': '5433',
        'database': 'moviedb',
        'user': 'demo',
        'password': 'demo123'
    },
    {
        'name': 'Docker Compose (postgres service)',
        'host': 'postgres',
        'port': '5432',
        'database': 'moviedb',
        'user': 'demo',
        'password': 'demo123'
    },
    {
        'name': 'Environment variables',
        'host': os.getenv('POSTGRES_HOST', 'localhost'),
        'port': os.getenv('POSTGRES_PORT', '5433'),
        'database': os.getenv('POSTGRES_DB', 'moviedb'),
        'user': os.getenv('POSTGRES_USER', 'demo'),
        'password': os.getenv('POSTGRES_PASSWORD', 'demo123')
    }
]

def test_connection(config):
    try:
        engine = create_engine(f"postgresql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/{config['database']}")
        # Test connection
        with engine.connect() as conn:
            result = conn.execute(text("SELECT 1"))
            return engine, True
    except Exception as e:
        return None, str(e)

# Try each configuration
engine = None
for config in DB_CONFIGS:
    print(f"Trying connection: {config['name']}")
    test_engine, result = test_connection(config)
    if test_engine:
        engine = test_engine
        print(f"✅ Connection successful with: {config['name']}")
        break
    else:
        print(f"❌ Failed: {result}")

if engine is None:
    print("\n⚠️  TROUBLESHOOTING STEPS:")
    print("1. Start the database using: docker-compose -f docker-compose.dev.yml up -d")
    print("2. Wait 10-15 seconds for PostgreSQL to fully start")
    print("3. Check if containers are running: docker-compose -f docker-compose.dev.yml ps")
    print("4. If still failing, check Docker logs: docker-compose -f docker-compose.dev.yml logs postgres")
    print("\n📝 Note: This notebook requires a running PostgreSQL database to function properly.")
    raise ConnectionError("Could not establish database connection. Please start the database first.")
else:
    print("\n🎉 Database connection established and ready for analysis!")


## 1. Database Schema Exploration


In [ ]:
# Get all table names
tables_query = """
SELECT table_name 
FROM information_schema.tables 
WHERE table_schema = 'public'
"""

tables = pd.read_sql(tables_query, engine)
print("Tables in the database:")
print(tables)


In [ ]:
# Get detailed schema information for each table
schema_query = """
SELECT 
    table_name,
    column_name,
    data_type,
    is_nullable,
    column_default
FROM information_schema.columns 
WHERE table_schema = 'public'
ORDER BY table_name, ordinal_position
"""

schema_info = pd.read_sql(schema_query, engine)
print("Database Schema:")
print(schema_info)


In [ ]:
# Get table row counts
row_counts = {}
for table in ['movies', 'users', 'ratings']:
    count_query = f"SELECT COUNT(*) as count FROM {table}"
    result = pd.read_sql(count_query, engine)
    row_counts[table] = result.iloc[0]['count']

print("Table Row Counts:")
for table, count in row_counts.items():
    print(f"{table}: {count} rows")


## 2. Movies Table Analysis


In [ ]:
# Load movies data
movies_df = pd.read_sql("SELECT * FROM movies", engine)
print("Movies Dataset:")
print(f"Shape: {movies_df.shape}")
print("\nFirst 5 rows:")
movies_df.head()


In [ ]:
# Movies data summary
print("Movies Data Summary:")
print(movies_df.describe())
print("\nData Types:")
print(movies_df.dtypes)
print("\nMissing Values:")
print(movies_df.isnull().sum())


In [ ]:
# Movie genre distribution
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
genre_counts = movies_df['genre_primary'].value_counts()
plt.pie(genre_counts.values, labels=genre_counts.index, autopct='%1.1f%%', startangle=90)
plt.title('Movie Genre Distribution')

plt.subplot(1, 2, 2)
sns.countplot(data=movies_df, y='genre_primary', order=genre_counts.index)
plt.title('Movies by Genre')
plt.xlabel('Count')

plt.tight_layout()
plt.show()

print("Genre Distribution:")
print(genre_counts)


In [ ]:
# Movie performance analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Budget vs Revenue
axes[0, 0].scatter(movies_df['budget_millions'], movies_df['revenue_millions'], alpha=0.7)
axes[0, 0].set_xlabel('Budget (Millions)')
axes[0, 0].set_ylabel('Revenue (Millions)')
axes[0, 0].set_title('Budget vs Revenue')
# Add diagonal line for break-even
max_val = max(movies_df['budget_millions'].max(), movies_df['revenue_millions'].max())
axes[0, 0].plot([0, max_val], [0, max_val], 'r--', alpha=0.5, label='Break-even line')
axes[0, 0].legend()

# Rating distribution
axes[0, 1].hist(movies_df['rating'], bins=10, alpha=0.7, edgecolor='black')
axes[0, 1].set_xlabel('Rating')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Movie Rating Distribution')

# Year vs Rating
axes[1, 0].scatter(movies_df['year'], movies_df['rating'], alpha=0.7)
axes[1, 0].set_xlabel('Year')
axes[1, 0].set_ylabel('Rating')
axes[1, 0].set_title('Movie Year vs Rating')

# Duration distribution
axes[1, 1].hist(movies_df['duration_minutes'], bins=8, alpha=0.7, edgecolor='black')
axes[1, 1].set_xlabel('Duration (Minutes)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Movie Duration Distribution')

plt.tight_layout()
plt.show()


In [ ]:
# Calculate ROI and profitability metrics
movies_df['roi'] = (movies_df['revenue_millions'] - movies_df['budget_millions']) / movies_df['budget_millions'] * 100
movies_df['profit_millions'] = movies_df['revenue_millions'] - movies_df['budget_millions']
movies_df['profitable'] = movies_df['profit_millions'] > 0

print("Movie Performance Metrics:")
print(f"Average ROI: {movies_df['roi'].mean():.1f}%")
print(f"Profitable movies: {movies_df['profitable'].sum()}/{len(movies_df)} ({movies_df['profitable'].mean()*100:.1f}%)")
print(f"Total profit across all movies: ${movies_df['profit_millions'].sum():.1f}M")

print("\nTop 5 movies by ROI:")
top_roi = movies_df.nlargest(5, 'roi')[['title', 'year', 'budget_millions', 'revenue_millions', 'roi']]
print(top_roi)

print("\nTop 5 movies by profit:")
top_profit = movies_df.nlargest(5, 'profit_millions')[['title', 'year', 'budget_millions', 'revenue_millions', 'profit_millions']]
print(top_profit)


## 3. Users Table Analysis


In [ ]:
# Load users data
users_df = pd.read_sql("SELECT * FROM users", engine)
print("Users Dataset:")
print(f"Shape: {users_df.shape}")
print("\nFirst 5 rows:")
users_df.head()


In [ ]:
# Users data analysis
print("Users Data Summary:")
print(users_df.describe())
print("\nMissing Values:")
print(users_df.isnull().sum())


In [ ]:
# User demographics analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Age distribution
axes[0, 0].hist(users_df['age'], bins=8, alpha=0.7, edgecolor='black')
axes[0, 0].set_xlabel('Age')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('User Age Distribution')

# Country distribution
country_counts = users_df['country'].value_counts()
axes[0, 1].pie(country_counts.values, labels=country_counts.index, autopct='%1.1f%%', startangle=90)
axes[0, 1].set_title('Users by Country')

# Subscription type distribution
sub_counts = users_df['subscription_type'].value_counts()
axes[1, 0].bar(sub_counts.index, sub_counts.values, alpha=0.7)
axes[1, 0].set_xlabel('Subscription Type')
axes[1, 0].set_ylabel('Count')
axes[1, 0].set_title('Subscription Type Distribution')

# Convert joined_date to datetime for analysis
users_df['joined_date'] = pd.to_datetime(users_df['joined_date'])
users_df['joined_month'] = users_df['joined_date'].dt.strftime('%Y-%m')
monthly_joins = users_df['joined_month'].value_counts().sort_index()
axes[1, 1].plot(monthly_joins.index, monthly_joins.values, marker='o')
axes[1, 1].set_xlabel('Month')
axes[1, 1].set_ylabel('New Users')
axes[1, 1].set_title('User Signups Over Time')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("User Demographics Summary:")
print(f"Average age: {users_df['age'].mean():.1f} years")
print(f"Age range: {users_df['age'].min()} - {users_df['age'].max()} years")
print(f"Premium users: {(users_df['subscription_type'] == 'premium').sum()}/{len(users_df)} ({(users_df['subscription_type'] == 'premium').mean()*100:.1f}%)")


## 4. Ratings Table Analysis


In [ ]:
# Load ratings data
ratings_df = pd.read_sql("SELECT * FROM ratings", engine)
print("Ratings Dataset:")
print(f"Shape: {ratings_df.shape}")
print("\nFirst 5 rows:")
ratings_df.head()


In [ ]:
# Ratings analysis
print("Ratings Data Summary:")
print(ratings_df.describe())
print("\nMissing Values:")
print(ratings_df.isnull().sum())
print("\nUnique users who rated:", ratings_df['user_id'].nunique())
print("Unique movies rated:", ratings_df['movie_id'].nunique())


In [ ]:
# Rating distribution analysis
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Rating distribution
rating_counts = ratings_df['rating'].value_counts().sort_index()
axes[0].bar(rating_counts.index, rating_counts.values, alpha=0.7)
axes[0].set_xlabel('Rating')
axes[0].set_ylabel('Frequency')
axes[0].set_title('User Rating Distribution')
axes[0].set_xticks(range(1, 11))

# Ratings per user
ratings_per_user = ratings_df['user_id'].value_counts()
axes[1].hist(ratings_per_user.values, bins=5, alpha=0.7, edgecolor='black')
axes[1].set_xlabel('Number of Ratings')
axes[1].set_ylabel('Number of Users')
axes[1].set_title('Ratings per User Distribution')

# Ratings per movie
ratings_per_movie = ratings_df['movie_id'].value_counts()
axes[2].hist(ratings_per_movie.values, bins=5, alpha=0.7, edgecolor='black')
axes[2].set_xlabel('Number of Ratings')
axes[2].set_ylabel('Number of Movies')
axes[2].set_title('Ratings per Movie Distribution')

plt.tight_layout()
plt.show()

print("Rating Statistics:")
print(f"Average rating: {ratings_df['rating'].mean():.2f}")
print(f"Most common rating: {ratings_df['rating'].mode()[0]}")
print(f"Average ratings per user: {ratings_per_user.mean():.1f}")
print(f"Average ratings per movie: {ratings_per_movie.mean():.1f}")


## 5. Cross-Table Analysis & Joins


In [ ]:
# Comprehensive join query
join_query = """
SELECT 
    m.title,
    m.year,
    m.genre_primary,
    m.rating as imdb_rating,
    m.budget_millions,
    m.revenue_millions,
    m.director,
    u.name as user_name,
    u.age,
    u.country,
    u.subscription_type,
    r.rating as user_rating,
    r.review_text,
    r.timestamp
FROM ratings r
JOIN movies m ON r.movie_id = m.id
JOIN users u ON r.user_id = u.id
ORDER BY r.timestamp
"""

full_data = pd.read_sql(join_query, engine)
print("Complete dataset with joins:")
print(f"Shape: {full_data.shape}")
full_data.head()


In [ ]:
# Compare IMDB ratings vs User ratings
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(full_data['imdb_rating'], full_data['user_rating'], alpha=0.7)
plt.xlabel('IMDB Rating')
plt.ylabel('User Rating')
plt.title('IMDB vs User Ratings')
# Add diagonal line
plt.plot([6, 10], [6, 10], 'r--', alpha=0.5, label='Perfect agreement')
plt.legend()

plt.subplot(1, 2, 2)
full_data['rating_diff'] = full_data['user_rating'] - full_data['imdb_rating']
plt.hist(full_data['rating_diff'], bins=10, alpha=0.7, edgecolor='black')
plt.xlabel('Rating Difference (User - IMDB)')
plt.ylabel('Frequency')
plt.title('Rating Difference Distribution')
plt.axvline(0, color='red', linestyle='--', alpha=0.5, label='No difference')
plt.legend()

plt.tight_layout()
plt.show()

print(f"Correlation between IMDB and User ratings: {full_data['imdb_rating'].corr(full_data['user_rating']):.3f}")
print(f"Average rating difference (User - IMDB): {full_data['rating_diff'].mean():.2f}")


In [ ]:
# User behavior analysis by demographics
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Average rating by age group
full_data['age_group'] = pd.cut(full_data['age'], bins=[0, 25, 30, 35, 100], labels=['<25', '25-30', '30-35', '35+'])
age_ratings = full_data.groupby('age_group')['user_rating'].mean()
axes[0, 0].bar(age_ratings.index, age_ratings.values, alpha=0.7)
axes[0, 0].set_ylabel('Average Rating')
axes[0, 0].set_title('Average Rating by Age Group')

# Average rating by country
country_ratings = full_data.groupby('country')['user_rating'].mean()
axes[0, 1].bar(country_ratings.index, country_ratings.values, alpha=0.7)
axes[0, 1].set_ylabel('Average Rating')
axes[0, 1].set_title('Average Rating by Country')
axes[0, 1].tick_params(axis='x', rotation=45)

# Average rating by subscription type
sub_ratings = full_data.groupby('subscription_type')['user_rating'].mean()
axes[1, 0].bar(sub_ratings.index, sub_ratings.values, alpha=0.7)
axes[1, 0].set_ylabel('Average Rating')
axes[1, 0].set_title('Average Rating by Subscription Type')

# Average rating by genre
genre_ratings = full_data.groupby('genre_primary')['user_rating'].mean()
axes[1, 1].bar(genre_ratings.index, genre_ratings.values, alpha=0.7)
axes[1, 1].set_ylabel('Average Rating')
axes[1, 1].set_title('Average Rating by Genre')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()


## 6. Advanced Analytics & Insights


In [ ]:
# Movie popularity and performance metrics
movie_metrics_query = """
SELECT 
    m.title,
    m.year,
    m.genre_primary,
    m.rating as imdb_rating,
    m.budget_millions,
    m.revenue_millions,
    COUNT(r.rating) as num_ratings,
    AVG(r.rating) as avg_user_rating,
    STDDEV(r.rating) as rating_stddev
FROM movies m
LEFT JOIN ratings r ON m.id = r.movie_id
GROUP BY m.id, m.title, m.year, m.genre_primary, m.rating, m.budget_millions, m.revenue_millions
ORDER BY num_ratings DESC, avg_user_rating DESC
"""

movie_metrics = pd.read_sql(movie_metrics_query, engine)
print("Movie Performance Metrics:")
movie_metrics.head(10)


In [ ]:
# User engagement analysis
user_engagement_query = """
SELECT 
    u.name,
    u.age,
    u.country,
    u.subscription_type,
    COUNT(r.rating) as num_ratings,
    AVG(r.rating) as avg_rating_given,
    STDDEV(r.rating) as rating_variance,
    MIN(r.timestamp) as first_rating,
    MAX(r.timestamp) as last_rating
FROM users u
LEFT JOIN ratings r ON u.id = r.user_id
GROUP BY u.id, u.name, u.age, u.country, u.subscription_type
ORDER BY num_ratings DESC
"""

user_engagement = pd.read_sql(user_engagement_query, engine)
print("User Engagement Metrics:")
user_engagement.head()


In [ ]:
# Genre preferences by user demographics
genre_prefs_query = """
SELECT 
    u.subscription_type,
    u.country,
    m.genre_primary,
    COUNT(*) as ratings_count,
    AVG(r.rating) as avg_rating
FROM ratings r
JOIN users u ON r.user_id = u.id
JOIN movies m ON r.movie_id = m.id
GROUP BY u.subscription_type, u.country, m.genre_primary
ORDER BY u.subscription_type, u.country, ratings_count DESC
"""

genre_prefs = pd.read_sql(genre_prefs_query, engine)
print("Genre Preferences by Demographics:")
genre_prefs.head(10)


In [ ]:
# Create a preference heatmap
pivot_data = genre_prefs.pivot_table(
    values='avg_rating', 
    index=['subscription_type', 'country'], 
    columns='genre_primary', 
    fill_value=0
)

plt.figure(figsize=(12, 8))
sns.heatmap(pivot_data, annot=True, cmap='YlOrRd', fmt='.1f', cbar_kws={'label': 'Average Rating'})
plt.title('Genre Preferences Heatmap (Average Rating by Demographics)')
plt.ylabel('Subscription Type & Country')
plt.xlabel('Genre')
plt.tight_layout()
plt.show()


## 7. Business Insights & Recommendations


In [ ]:
# Key business metrics and insights
print("=== MOVIEDB DATABASE INSIGHTS ===")
print()

print("📊 DATABASE OVERVIEW:")
print(f"• {len(movies_df)} movies in catalog")
print(f"• {len(users_df)} registered users")
print(f"• {len(ratings_df)} total ratings submitted")
print(f"• {ratings_df['movie_id'].nunique()}/{len(movies_df)} movies have been rated")
print()

print("🎬 MOVIE INSIGHTS:")
print(f"• Most popular genre: {movies_df['genre_primary'].mode()[0]} ({movies_df['genre_primary'].value_counts().iloc[0]} movies)")
print(f"• Average movie rating: {movies_df['rating'].mean():.1f}/10")
print(f"• Most profitable movie: {movies_df.loc[movies_df['profit_millions'].idxmax(), 'title']} (${movies_df['profit_millions'].max():.0f}M profit)")
print(f"• Best ROI: {movies_df.loc[movies_df['roi'].idxmax(), 'title']} ({movies_df['roi'].max():.0f}% ROI)")
print()

print("👥 USER INSIGHTS:")
print(f"• Average user age: {users_df['age'].mean():.1f} years")
print(f"• Premium subscribers: {(users_df['subscription_type'] == 'premium').mean()*100:.0f}%")
print(f"• Most active country: {users_df['country'].mode()[0]}")
print(f"• Average ratings per user: {ratings_df['user_id'].value_counts().mean():.1f}")
print()

print("⭐ RATING INSIGHTS:")
print(f"• Average user rating: {ratings_df['rating'].mean():.1f}/10")
print(f"• Most common rating: {ratings_df['rating'].mode()[0]}/10")
print(f"• User vs IMDB rating correlation: {full_data['imdb_rating'].corr(full_data['user_rating']):.2f}")
print(f"• Users tend to rate {abs(full_data['rating_diff'].mean()):.1f} points {'higher' if full_data['rating_diff'].mean() > 0 else 'lower'} than IMDB")
print()

print("💡 RECOMMENDATIONS:")
print("• Focus on Action movies - highest representation and good ratings")
print("• Premium users show higher engagement - consider targeted content")
print("• Users rate slightly different from IMDB - consider user-based recommendations")
print("• Some movies lack user ratings - implement rating incentives")


In [ ]:
# Export key findings to CSV for further analysis
print("Exporting analysis results...")

# Export enhanced movie data
movies_enhanced = movies_df.copy()
movies_enhanced.to_csv('movies_analysis.csv', index=False)

# Export user engagement metrics
user_engagement.to_csv('user_engagement_analysis.csv', index=False)

# Export movie performance metrics
movie_metrics.to_csv('movie_performance_analysis.csv', index=False)

print("Analysis complete! Key files exported:")
print("• movies_analysis.csv - Enhanced movie data with calculated metrics")
print("• user_engagement_analysis.csv - User engagement and behavior metrics")
print("• movie_performance_analysis.csv - Movie popularity and rating statistics")
